<a href="https://colab.research.google.com/github/thavha/Regression-predict-group-5/blob/master/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Preprocessing**
Importing Liabraries


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

**Importing the data set** and checking the data

In [0]:
dataset = pd.read_csv('Train.csv')
###x =dataset.iloc[:,:-1].values
###y = dataset.iloc[:,-1].values

In [5]:
dataset.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


selecting only the rlevent  columns that will be used for the model and replacing white space with _ on columns names

In [0]:
new_df = dataset[['User Id','Platform Type','Personal or Business','Placement - Day of Month','Placement - Weekday (Mo = 1)','Placement - Time','Confirmation - Time','Arrival at Pickup - Time','Arrival at Destination - Time','Distance (KM)','Temperature','Rider Id','Time from Pickup to Arrival']].copy()
new_df.columns = [col.replace(" ","_") for col in new_df.columns] 


In [24]:
new_df.head()

,User_Id,Platform_Type,Personal_or_Business,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival
0,User_Id_633,3,Business,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745
1,User_Id_2285,3,Personal,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993
2,User_Id_265,3,Business,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,NaN,Rider_Id_155,455
3,User_Id_1402,3,Business,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341
4,User_Id_1737,1,Personal,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214


encoding personal or business column


In [25]:
new_df = pd.get_dummies(new_df, columns=['Personal_or_Business'], prefix = ['personal'])
new_df = new_df.iloc[:, :-1]
new_df.columns = [*new_df.columns[:-1], 'Business']
new_df.head()

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business
0,User_Id_633,3,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,1
1,User_Id_2285,3,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,0
2,User_Id_265,3,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,NaN,Rider_Id_155,455,1
3,User_Id_1402,3,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,1
4,User_Id_1737,1,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,0


In [26]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   User_Id                        21201 non-null  object 
 1   Platform_Type                  21201 non-null  int64  
 2   Placement_-_Day_of_Month       21201 non-null  int64  
 3   Placement_-_Weekday_(Mo_=_1)   21201 non-null  int64  
 4   Placement_-_Time               21201 non-null  object 
 5   Confirmation_-_Time            21201 non-null  object 
 6   Arrival_at_Pickup_-_Time       21201 non-null  object 
 7   Arrival_at_Destination_-_Time  21201 non-null  object 
 8   Distance_(KM)                  21201 non-null  int64  
 9   Temperature                    16835 non-null  float64
 10  Rider_Id                       21201 non-null  object 
 11  Time_from_Pickup_to_Arrival    21201 non-null  int64  
 12  Business                       21201 non-null 

dealing with the missing valuse replace with the mean/median temperature 

In [29]:
new_df['Temperature'] = new_df.Temperature.fillna(new_df.Temperature.median())
new_df.head()

,User_Id,Platform_Type,Placement_-_Day_of_Month,Placement_-_Weekday_(Mo_=_1),Placement_-_Time,Confirmation_-_Time,Arrival_at_Pickup_-_Time,Arrival_at_Destination_-_Time,Distance_(KM),Temperature,Rider_Id,Time_from_Pickup_to_Arrival,Business
0,User_Id_633,3,9,5,9:35:46 AM,9:40:10 AM,10:04:47 AM,10:39:55 AM,4,20.4,Rider_Id_432,745,1
1,User_Id_2285,3,12,5,11:16:16 AM,11:23:21 AM,11:40:22 AM,12:17:22 PM,16,26.4,Rider_Id_856,1993,0
2,User_Id_265,3,30,2,12:39:25 PM,12:42:44 PM,12:49:34 PM,1:00:38 PM,3,23.5,Rider_Id_155,455,1
3,User_Id_1402,3,15,5,9:25:34 AM,9:26:05 AM,9:37:56 AM,10:05:27 AM,9,19.2,Rider_Id_855,1341,1
4,User_Id_1737,1,13,1,9:55:18 AM,9:56:18 AM,10:03:53 AM,10:25:37 AM,9,15.4,Rider_Id_770,1214,0
